#### Scenario
A healthcare provider initiated an attrition analysis. They wish to know their current attrition rate and, if the rate is greater than ten percent, what are the contributing factors and potential solutions.

#### Problem Statement
How can we increase employee retention while minimizing costs?

In [11]:
from pathlib import Path
from math import ceil

import pandas as pd
from pandasql import sqldf

import utils

P_SQL = lambda q: sqldf(q, globals())

In [16]:
# Import file
filename = 'data/healthcare_attrition.csv'
df = pd.read_csv(Path(utils.root(), filename), sep=',')

# Constants
EMPLOYED_SAMPLE_DF = df.query("Attrition == 'No'")
ATTRITION_SAMPLE_DF = df.query("Attrition == 'Yes'")

EMPLOYED_SAMPLE_SIZE = EMPLOYED_SAMPLE_DF.shape[0]
ATTRITION_SAMPLE_SIZE = ATTRITION_SAMPLE_DF.shape[0]

In [21]:
sr = df['BusinessTravel']
shape_ = sr.size
sr2 = sr.where(sr == 'Travel_Frequently').value_counts() / shape_
print(sr2)
print(shape_)

Travel_Frequently    0.190931
Name: BusinessTravel, dtype: float64
1676


In [3]:
# ATTRITION RATE
attrition_sr = df['Attrition'] # isolating Attrition column

attrition_count = attrition_sr.value_counts()['Yes']
num_of_employees = attrition_sr.count()

attrition_rate_percentage = round(attrition_count / num_of_employees, 2) * 100
print(f'The attrition rate is approximately {attrition_rate_percentage}%')

Attrition Rate: 13.47%


In [4]:
# ATTRITION BY AGE
attrition_df = df.query("Attrition == 'Yes'")
attrition_population_size = ATTRITION_SAMPLE_DF['EmployeeID'].count()

attrition_age_sr = ATTRITION_SAMPLE_DF['Age'] # Isolate age column

# NOTE: The empirical rule states that 68% of data are within 1 standard deviation from the mean. The following code counts the number of data points within 1 standard deviation from the mean and divides it by the attrition count (population size)--the actual percentage of attrition that occurs within 1 standard deviation of the mean.
empirical_rule_68 = utils.stats(attrition_age_sr)['empirical_rule_68']

# Counts of data points within 1 standard deviation of the mean.
one_stddev_count = 0
for i in attrition_age_sr:
  if i >= empirical_rule_68[0] and i <= empirical_rule_68[1]:
    one_stddev_count += 1

# Percentage of data points that fall within 1 standard deviation of mean
one_stddev_percentage = round((one_stddev_count / ATTRITION_SAMPLE_SIZE) * 100, 2)

print(f'{one_stddev_percentage}% of attrition occurs between ages {empirical_rule_68[0]} and {empirical_rule_68[1]} (1 standard dev from mean).')

72.86% of attrition occurs between ages 22 and 40 (1 standard dev from mean).


In [8]:
# ATTRITION BY TRAVEL

# calc two tailed z test on two proportions = https://www.youtube.com/watch?v=pCbNUnZ98oE
def observed_proportion(sr: pd.Series, n: int) -> pd.DataFrame:
    COLNAME = 'Observed_Proportion'
    
    sr_counts = sr.value_counts()
    df = sr_counts.to_frame(name='Count').reset_index().rename(columns={'index': 'Travel_Type'})
    df[COLNAME] = None
    
    for row in df.itertuples():
        percentage = row.Count / n
        df.at[row.Index, COLNAME] = percentage 
    return df

attrition_travel_proportion_df = observed_proportion(ATTRITION_SAMPLE_DF['BusinessTravel'], ATTRITION_SAMPLE_SIZE)

employed_travel_proportion_df = observed_proportion(EMPLOYED_SAMPLE_DF['BusinessTravel'], EMPLOYED_SAMPLE_SIZE)


print(employed_travel_proportion_df)
print(attrition_travel_proportion_df)

# Ho: p1 = p2 
# Ha: p1 != p2

# n1 (attrition) = 199 
# x1 = 57

# n2 (employed) = 1477
# x2 = 263

# p1 (sample proportion) = x1 / n1 = 57 / 199 = 0.286
# p2 (sample proportion) = x2 / n2 = 263 / 1477 = 0.178

# alpha = 0.05
# alpha / 2 = 0.025

# z-value = 1.96 (always for alpha 0.05)

# calculate p = pooled_proportion
# (x1+x2)/(n1+n2) = (57+263)/(199+1477) = 320/1676 = 0.1909

# calculate z-score
# z = (p1-p2) / sqrt[p(1-p)((1/n1)+(1/n2))]
# z = (0.286-0.178) / sqrt[0.1909(1-0.1909)((1/199)+(1/1477))]
# z = 0.108 / sqrt[0.1546(0.0057)]
# z = 0.108 / sqrt[0.0009]
# z = 0.108 / 0.03
# z = 3.6

# the difference is statistically significant


         Travel_Type  Count Percentage_of_Observations
0      Travel_Rarely   1058                   0.716317
1  Travel_Frequently    263                   0.178064
2         Non-Travel    156                   0.105619
         Travel_Type  Count Percentage_of_Observations
0      Travel_Rarely    126                   0.633166
1  Travel_Frequently     57                   0.286432
2         Non-Travel     16                   0.080402
